In [3]:
from main import profile_run
from utillities import check_particle_collision, execute_collision, run_loop, push_next_collision
%load_ext line_profiler

In [4]:
%lprun -f profile_run profile_run()

Placing particles
Finding inital collisions
Time elapsed: 241.02854561805725


Timer unit: 1e-06 s

Total time: 241.134 s
File: /home/martin/repos/test/Exercises_CompPhys/ex1/main.py
Function: profile_run at line 76

Line #      Hits         Time  Per Hit   % Time  Line Contents
    76                                           def profile_run():
    77                                               # https://web.archive.org/web/20140513005858im_/http://www.appneta.com/blog/line-profiler-python/
    78         1          2.0      2.0      0.0      name = "profile_run"
    79         1       3803.0   3803.0      0.0      xi, N, T, R = read_params(para_dir + name)
    80         1        159.0    159.0      0.0      radii = np.ones(N) * R
    81         1         10.0     10.0      0.0      masses = np.ones(N)
    82         1          1.0      1.0      0.0      args = (N, T, radii, masses, xi)
    83         1  241130314.0 241130314.0    100.0      particles, t = run_loop(random_dist, args)

In [5]:
%lprun -f run_loop profile_run()

Placing particles
Finding inital collisions
Time elapsed: 279.77443742752075


Timer unit: 1e-06 s

Total time: 279.195 s
File: /home/martin/repos/test/Exercises_CompPhys/ex1/utillities.py
Function: run_loop at line 228

Line #      Hits         Time  Per Hit   % Time  Line Contents
   228                                           def run_loop(init, args, condition=None, n_check=np.inf, TC=False):
   229         1         10.0     10.0      0.0      tic = time.time()
   230         1   17329310.0 17329310.0      6.2      t, particles, collisions, last_collided = setup_loop(init, args)
   231         1          2.0      2.0      0.0      N, T, radii, masses, xi = args
   232                                           
   233         1          1.0      1.0      0.0      n = 0
   234         1         73.0     73.0      0.0      bar = Bar("running simulation", max=T)
   235    118081      88047.0      0.7      0.0      while n < T:
   236    118080    1679596.0     14.2      0.6          col = get_next_col(collisions)
   237    118080     154122.0      1.3      0.1 

In [6]:
%lprun -f execute_collision profile_run()

Placing particles
Finding inital collisions
Time elapsed: 306.6140718460083


Timer unit: 1e-06 s

Total time: 287.49 s
File: /home/martin/repos/test/Exercises_CompPhys/ex1/utillities.py
Function: execute_collision at line 205

Line #      Hits         Time  Per Hit   % Time  Line Contents
   205                                           def execute_collision(n, t, particles, collisions, last_collided, args, col, TC):
   206     10000      13325.0      1.3      0.0      N, T, radii, masses, xi = args
   207     10000       7364.0      0.7      0.0      t_next, i, j, t_added, col_type  = col
   208     10000     726089.0     72.6      0.3      particles[n+1] = particles[n]
   209     10000      25166.0      2.5      0.0      dt = col[0] - t[n]
   210     10000      12377.0      1.2      0.0      t[n+1] = t_next
   211                                           
   212     10000       6668.0      0.7      0.0      if TC:
   213                                                   xi = tc_check(i, n, t, last_collided, xi)
   214                                         

In [7]:
%lprun -f push_next_collision profile_run()

Placing particles
Finding inital collisions
Time elapsed: 364.6188898086548


Timer unit: 1e-06 s

Total time: 321.276 s
File: /home/martin/repos/test/Exercises_CompPhys/ex1/utillities.py
Function: push_next_collision at line 23

Line #      Hits         Time  Per Hit   % Time  Line Contents
    23                                           def push_next_collision(particles, n, i, t, collisions, radii):
    24     20425     140770.0      6.9      0.0      wall0 = check_wall_collison(particles[n, i, 0], particles[n, i, 2], radii[i])
    25     20425      77511.0      3.8      0.0      wall1 = check_wall_collison(particles[n, i, 1], particles[n, i, 3], radii[i])
    26     20425      89938.0      4.4      0.0      if wall0 != np.inf: heapq.heappush(collisions, (t+wall0, i, -1, t, "wall0"))
    27     20425      50118.0      2.5      0.0      if wall1 != np.inf: heapq.heappush(collisions, (t+wall1, i, -1, t, "wall1"))
    28  20445425   11610429.0      0.6      3.6      for j in range(len(particles[0])):
    29  20425000  290921397.0     14.2     90.6          dt = 

In [8]:
%lprun -f check_particle_collision profile_run()

Placing particles
Finding inital collisions
Time elapsed: 375.2568097114563


Timer unit: 1e-06 s

Total time: 290.441 s
File: /home/martin/repos/test/Exercises_CompPhys/ex1/utillities.py
Function: check_particle_collision at line 104

Line #      Hits         Time  Per Hit   % Time  Line Contents
   104                                           def check_particle_collision(particles, n, i, j, radii):
   105  20394000   20972894.0      1.0      7.2      R = radii[i] + radii[j]
   106  20394000   51333507.0      2.5     17.7      dx = particles[n, j, :2] - particles[n, i, :2]
   107  20394000   48744419.0      2.4     16.8      dv = particles[n, j, 2:] - particles[n, i, 2:]
   108  20394000  143744104.0      7.0     49.5      d = (dv @ dx)**2 - (dv @ dv) * ((dx @ dx) - R**2)
   109  20394000   22797522.0      1.1      7.8      if (d <= 0 or dv @ dx >= 0): return np.inf
   110    306955    2849047.0      9.3      1.0      else: return - (dv @ dx + np.sqrt(d)) / (dv @ dv)